In [55]:
import pandas as pd
import requests

# Téléchargement de la base de données:

In [56]:
url = "https://raw.githubusercontent.com/WildCodeSchool/data-training-resources/main/quests/Market_retail.csv"  
response = requests.get(url)

if response.status_code == 200:
    with open("market_price_retail.csv", "wb") as file:
        file.write(response.content)
    print("Fichier téléchargé avec succès !")
else:
    print(f"Erreur : {response.status_code}")

Fichier téléchargé avec succès !


# Exploration/Nettoyage des données

In [188]:
df = pd.read_csv("./donnees/brutes/market_price_retail.csv")
df

,Order ID,Order Date,Customer Name,Country,State,City,Region,Segment,Ship Mode,Category,Sub-Category,Product Name,Discount,Sales,Profit,Quantity,Feedback?
0,BN-2011-7407039,1/1/2011,Ruby Patel,Sweden,Stockholm,Stockholm,North,Home Office,Economy Plus,Office Supplies,Paper,"Enermax Note Cards, Premium",$ 0.5,$ 45,$ -26,3,False
1,AZ-2011-9050313,1/3/2011,Summer Hayward,United Kingdom,England,Southport,North,Consumer,Economy,Furniture,Bookcases,"Dania Corner Shelving, Traditional",$ -,$ 854,$ 290,7,True
2,AZ-2011-6674300,1/4/2011,Devin Huddleston,France,Auvergne-Rhône-Alpes,Valence,Central,Consumer,Economy,Office Supplies,Art,"Binney & Smith Sketch Pad, Easy-Erase",$ -,$ 140,$ 21,3,True
3,BN-2011-2819714,1/4/2011,Mary Parker,United Kingdom,England,Birmingham,North,Corporate,Economy,Office Supplies,Art,"Boston Markers, Easy-Erase",$ 0.5,$ 27,$ -22,2,True
4,BN-2011-2819714,1/4/2011,Mary Parker,United Kingdom,England,Birmingham,North,Corporate,Economy,Office Supplies,Storage,"Eldon Folders, Single Width",$ 0.5,$ 17,$ -1,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8042,AZ-2014-766953,12/31/2014,Jose Gambino,United Kingdom,England,Maidenhead,North,Corporate,Economy,Furniture,Bookcases,"Ikea Stackable Bookrack, Traditional",$ -,$ 245,$ 91,2,True
8043,BN-2014-4140795,12/31/2014,Daniel Hamilton,Netherlands,North Brabant,Eindhoven,Central,Home Office,Economy Plus,Office Supplies,Art,"BIC Pencil Sharpener, Fluorescent",$ 0.5,$ 30,$ -10,2,False
8044,BN-2014-4140795,12/31/2014,Daniel Hamilton,Netherlands,North Brabant,Eindhoven,Central,Home Office,Economy Plus,Office Supplies,Binders,"Avery Binder Covers, Recycled",$ 0.5,$ 23,$ -6,4,True
8045,BN-2014-4140795,12/31/2014,Daniel Hamilton,Netherlands,North Brabant,Eindhoven,Central,Home Office,Economy Plus,Technology,Machines,"StarTech Phone, Red",$ 0.5,$ 108,$ -19,3,False


In [189]:
df.columns

Index(['Order ID', 'Order Date', 'Customer Name', 'Country', 'State', 'City',
       'Region', 'Segment', 'Ship Mode', 'Category', 'Sub-Category',
       'Product Name', ' Discount ', ' Sales ', ' Profit ', 'Quantity',
       'Feedback?'],
      dtype='object')

In [190]:
df.info()#Pas de valeurs null mais les types sont à revoir

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8047 entries, 0 to 8046
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order ID       8047 non-null   object
 1   Order Date     8047 non-null   object
 2   Customer Name  8047 non-null   object
 3   Country        8047 non-null   object
 4   State          8047 non-null   object
 5   City           8047 non-null   object
 6   Region         8047 non-null   object
 7   Segment        8047 non-null   object
 8   Ship Mode      8047 non-null   object
 9   Category       8047 non-null   object
 10  Sub-Category   8047 non-null   object
 11  Product Name   8047 non-null   object
 12   Discount      8047 non-null   object
 13   Sales         8047 non-null   object
 14   Profit        8047 non-null   object
 15  Quantity       8047 non-null   int64 
 16  Feedback?      8047 non-null   bool  
dtypes: bool(1), int64(1), object(15)
memory usage: 1013.9+ KB


In [191]:
#Tranformation du type de la colonne date:
df['Order Date'] = pd.to_datetime(df['Order Date'])

In [192]:
#On va modifier les colonnes 'Sales' 'Profit' et 'Discount' pour les transformer en float et supprimer les espaces dans les titres:
df['Sales'] = df[' Sales '].apply(lambda x : float(x[3:-1].replace(",", "")))
df['Discount'] = df[' Discount '].apply(lambda x : float(x[2:]) if '-' not in x else 0)
df['Profit'] = df[' Profit '].apply(lambda x : float(x[3:-1].replace(",", "")) if x[-2] != " " else 0)
df = df.drop([' Sales ', ' Discount ', ' Profit '], axis = 1)

In [ ]:
# Nettoyage terminé, on peut exporter ce df pour ervie de table de faits:
df.to_csv('./donnees/nettoyees/clean_FACT.csv', index=False)

# Création des tables de dimensions pour Order, Customer, Product

## DIM orders

In [ ]:
# Pas de fonction spéciale à faire, on séléctionne les colonnes dont les info ont spécifique à Order ID puis on drop_duplicates
df_order = df[['Order ID', 'Order Date', 'Customer Name', 'Country', 'State', 'City', 'Region', 'Ship Mode']].drop_duplicates().reset_index().drop('index', axis =1)
df_order

,Order ID,Order Date,Customer Name,Country,State,City,Region,Ship Mode
0,BN-2011-7407039,2011-01-01,Ruby Patel,Sweden,Stockholm,Stockholm,North,Economy Plus
1,AZ-2011-9050313,2011-01-03,Summer Hayward,United Kingdom,England,Southport,North,Economy
2,AZ-2011-6674300,2011-01-04,Devin Huddleston,France,Auvergne-Rhône-Alpes,Valence,Central,Economy
3,BN-2011-2819714,2011-01-04,Mary Parker,United Kingdom,England,Birmingham,North,Economy
4,AZ-2011-617423,2011-01-05,Daniel Burke,France,Auvergne-Rhône-Alpes,Echirolles,Central,Priority
...,...,...,...,...,...,...,...,...
4112,AZ-2014-4217323,2014-12-31,Evie Morton,France,Normandy,Caen,Central,Economy Plus
4113,AZ-2014-8174835,2014-12-31,Eloise Sykes,Germany,North Rhine-Westphalia,Bielefeld,Central,Economy
4114,AZ-2014-7604524,2014-12-31,Rebecca Chamberlain,Germany,Hamburg,Hamburg,Central,Economy
4115,AZ-2014-766953,2014-12-31,Jose Gambino,United Kingdom,England,Maidenhead,North,Economy


In [119]:
#On a bien que des valeurs uniques
df_order['Order ID'].value_counts()

Order ID
BN-2014-4140795    1
BN-2011-7407039    1
AZ-2011-9050313    1
AZ-2011-6674300    1
BN-2011-2819714    1
                  ..
AZ-2011-6439906    1
AZ-2011-4827146    1
AZ-2011-6712797    1
BN-2011-3248724    1
AZ-2011-2918397    1
Name: count, Length: 4117, dtype: int64

## DIM Customers

In [102]:
# Pas de fonction spéciale à faire, on séléctionne les colonnes dont les info sont spécifique à Customer Name puis on drop_duplicates
df_customer = df[['Customer Name', 'Segment']].drop_duplicates().reset_index().drop('index', axis = 1)
df_customer

,Customer Name,Segment
0,Ruby Patel,Home Office
1,Summer Hayward,Consumer
2,Devin Huddleston,Consumer
3,Mary Parker,Corporate
4,Daniel Burke,Home Office
...,...,...
787,Bethany Powell,Home Office
788,Alexandra Mahmood,Consumer
789,Carol Leach,Home Office
790,Charles Rice,Corporate


In [ ]:
#On a bien que des valeurs uniques
df_customer['Customer Name'].value_counts()

## DIM Product

In [183]:
# Plus compliqué, on veut ajouter une colonne 'Prix Unitaire'
# On fait déjà une copie du df avec les colonnes spécifiques à 'Product Name' + de quoi calculer 'Unit Price'
df_product = df[['Product Name', 'Category', 'Sub-Category', 'Sales', 'Quantity', 'Discount']].copy()

#On intègre le calcul pour calculer 'Unit Price"
df_product['Unit Price'] = round(df_product['Sales']/(df_product['Quantity'] * (1 - df_product['Discount'])), 2)

In [184]:
# On observe que pour un même produit, le 'Unit Price' varie légèrement, ce qui peut être expliqué par les différents arrondis 
# qu'on pu ubir les prix.
df_product[df_product['Product Name'] == "Eldon File Cart, Single Width"]


,Product Name,Category,Sub-Category,Sales,Quantity,Discount,Unit Price
177,"Eldon File Cart, Single Width",Office Supplies,Storage,230.0,2,0.1,127.78
203,"Eldon File Cart, Single Width",Office Supplies,Storage,230.0,2,0.1,127.78
266,"Eldon File Cart, Single Width",Office Supplies,Storage,576.0,5,0.1,128.00
1087,"Eldon File Cart, Single Width",Office Supplies,Storage,770.0,10,0.4,128.33
1520,"Eldon File Cart, Single Width",Office Supplies,Storage,257.0,2,0.0,128.50
1637,"Eldon File Cart, Single Width",Office Supplies,Storage,448.0,7,0.5,128.00
1727,"Eldon File Cart, Single Width",Office Supplies,Storage,384.0,3,0.0,128.00
1764,"Eldon File Cart, Single Width",Office Supplies,Storage,578.0,5,0.1,128.44
1765,"Eldon File Cart, Single Width",Office Supplies,Storage,128.0,1,0.0,128.00
2033,"Eldon File Cart, Single Width",Office Supplies,Storage,154.0,2,0.4,128.33


In [185]:
# On fait donc un groupby avec une moyenne de la colonne 'Unit Price' pour s'approcher au mieux de la vraie valeur, pui on arrondi à 2 décimales:
df_product = df_product.groupby(['Product Name', 'Category', 'Sub-Category'])[['Unit Price']].mean().reset_index()
df_product['Unit Price'] = round(df_product['Unit Price'], 2)
df_product

,Product Name,Category,Sub-Category,Unit Price
0,"Acco 3-Hole Punch, Clear",Office Supplies,Binders,29.65
1,"Acco 3-Hole Punch, Durable",Office Supplies,Binders,31.58
2,"Acco 3-Hole Punch, Economy",Office Supplies,Binders,30.07
3,"Acco 3-Hole Punch, Recycled",Office Supplies,Binders,30.46
4,"Acco Binder Covers, Clear",Office Supplies,Binders,12.90
...,...,...,...,...
1805,"Xerox Note Cards, Premium",Office Supplies,Paper,26.30
1806,"Xerox Note Cards, Recycled",Office Supplies,Paper,26.08
1807,"Xerox Parchment Paper, 8.5 x 11",Office Supplies,Paper,17.50
1808,"Xerox Parchment Paper, Premium",Office Supplies,Paper,13.36


# Exportation pour PowerBI

In [110]:
df_product.to_csv('./donnees/nettoyees/product_DIM.csv', index=False)
df_customer.to_csv('./donnees/nettoyees/customer_DIM.csv', index=False)
df_order.to_csv('./donnees/nettoyees/orders_DIM.csv', index=False)

In [193]:
df['Order Date'].min()

Timestamp('2011-01-01 00:00:00')

In [194]:
df['Order Date'].max()

Timestamp('2014-12-31 00:00:00')